# 高效的泡利噪声学习方法

泡利噪声学习的协议可以概括如下步骤：
1. 选择一系列逐渐增加的深度 $d = [m_0,m_1,...,m_n]$，即深度增加时，错误概率也增加，正确概率减小。
2. 为每个深度 $d$ 生成多个随机序列，其中每个门表示一个随机的克利福德门。当 $d=m$ 时，序列为 $[C^{-1}, C_m, C_{m-1}, \dots, C_2, C_1]$。
3. 对于每个序列，运行电路 $N$ 次并进行测量，得到测量结果概率分布 $q(m)$。
4. 通过 Walsh-Hadamard 变换，将结果概率分布 $q(m)$ 转换为泡利特征值。
5. 利用上述指数衰减模型，采用最小二乘法等拟合方法，拟合对应的深度 $m$ 以及其对应的泡利特征值 $\lambda_j(m)$，从而得到 $A_0$、$B_0$以及衰减参数 $\lambda_j$。
6. 再次通过逆 Walsh-Hadamard 变换，将泡利特征值 $\lambda$ 转换为对应的泡利错误率。

参考论文：Harper, Robin, et al. “Efficient Learning of Quantum Noise.” Nature Physics, vol. 16, no. 12, Dec. 2020, pp. 1184–88. arXiv.org, https://doi.org/10.1038/s41567-020-0992-8.

### 步骤一：构建泡利噪声模型

In [1]:
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error

# noise model
# Example error probabilities
p_reset = 0.03
p_meas = 0.01

p_gate_x = 0.001
p_gate_y = 0.000
p_gate_z = 0.0000

# p_two_gate = 0.1
# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate_x), ('Y',p_gate_y), ('Z',p_gate_z), ('I', 1 - (p_gate_x+p_gate_y+p_gate_z))])

error_gate2 = pauli_error([('X',p_gate_x), ('Y',p_gate_y), ('Z',p_gate_z), ('I', 1 - (p_gate_x+p_gate_y+p_gate_z))])
error_two_gate = error_gate1.tensor(error_gate1)

# Add errors to noise model
pauli_noise_model= NoiseModel()
pauli_noise_model.add_all_qubit_quantum_error(error_reset, "reset")
pauli_noise_model.add_all_qubit_quantum_error(error_meas, "measure")
pauli_noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
pauli_noise_model.add_all_qubit_quantum_error(error_two_gate, ["cx"])
sim_noise = AerSimulator(noise_model = pauli_noise_model)


### 步骤二：初始化量子噪声学习参数，运行泡利噪声学习算法。

初始化参数，我们考虑两个qubit，其中每次的clifford门数量为100个，暂时其中都为单clifford门，电路重复运行次数为1024次。
同时随机clifford电路的深度由1到15逐渐增加。

In [2]:
list_m = range(1,15)
num_samples = 100
# only consider single qubit twirl
num_1_qubit_twirl = num_samples
num_2_qubit_twirl = int(num_samples - num_1_qubit_twirl)
num_qubits = 2
shots = 1024
seed = 1010

运行泡利噪声学习协议。

In [3]:
from pauli_noise_learning import random_pauli_benchmarking

circuit_list, result_list, statistics_counts_list = random_pauli_benchmarking(num_qubits=num_qubits, rep_m_list = list_m, 
                                                                              num_1_qubit_twirl=num_1_qubit_twirl, num_2_qubit_twirl=num_2_qubit_twirl,
                                                                              shots = shots, sim = sim_noise)

now is running m = 1 experement
now is running m = 2 experement
now is running m = 3 experement
now is running m = 4 experement
now is running m = 5 experement
now is running m = 6 experement
now is running m = 7 experement
now is running m = 8 experement
now is running m = 9 experement
now is running m = 10 experement
now is running m = 11 experement
now is running m = 12 experement
now is running m = 13 experement
now is running m = 14 experement


### 步骤三：处理泡利噪声学习电路测量值
处理泡利噪声学习得到的测量值，进行拟合得到与比特上运行clifford门错误率有关的特征值。

In [4]:
from pauli_noise_learning import fit_iws_lambda
from sympy import fwht

#逆 walsh hadamard 变换，同时进行拟合。
lambda_list = fit_iws_lambda(list_m, statistics_counts_list, num_qubits, num_samples, shots)

#walsh hadamard 变换
eigenvalue_list = fwht(lambda_list)
print ("特征值: ", eigenvalue_list)

特征值:  [3.98834900330765, 0.00589927016169212, 0.00575527566549394, -3.54913483324726e-6]


上述泡利特征值（但只存在两个维度，出现泡利错误和不出现泡利错误），与作用一个clifford门引入噪声导致出错概率相关。

其中第一个表示都不出错的特征值为3.98802035751038，第二个表示第一个qubit出错的特征值为0.00587375657337752，第三个表示第二个qubit出错的特征值为0.00609302942369494，第四个表示两个qubit都出错的特征值为1.28564925481189e-5。

将特征值转换为概率的形式，并且将概率转换为作用一个clifford门之后，量子比特对应的出错率。

In [5]:
from pauli_noise_learning import eigenvalue_list_to_pauli_probability, pauli_probability_to_qubit_error_rate

probabilities = eigenvalue_list_to_pauli_probability(eigenvalue_list)

qubit_error_rates = pauli_probability_to_qubit_error_rate(num_qubits, probabilities)

In [6]:
print ("泡利噪声学习所获取的特征值: ", lambda_list)
print("特征值转化成的对应概率: ",probabilities)
print("量子比特的出错率（边缘概率）: ", qubit_error_rates)

泡利噪声学习所获取的特征值:  [1.0, 0.9970521394865706, 0.9971241367346697, 0.994172727086407]
特征值转化成的对应概率:  [0.9970846 0.0014757 0.0014397 0.       ]
量子比特的出错率（边缘概率）:  [0.0014756995866745345, 0.0014397010903885081]


如上所示，我们可以获得作用某个clifford门之后，平均的量子比特错误率为[0.0014756995866745345, 0.0014397010903885081]。